In [2]:
import pandas as pd

disney = pd.read_csv('/Users/student/Desktop/Data_Exploration_Project/1_Datenset/erstellte/cleaned/disney_plus_titles_cleaned.csv')
netflix = pd.read_csv('/Users/student/Desktop/Data_Exploration_Project/1_Datenset/erstellte/cleaned/netflix_titles_cleaned.csv')

agerating_map = { "G": 0, "TV-Y": 0, "TV-G": 0, "PG": 10, "TV-Y7": 7, "TV-Y7-FV": 7, "TV-PG": 10, "PG-13": 13, "R": 17, "TV-14": 14, "NC-17": 18, "TV-MA": 17, "NR": -1, "UR": -1 } 
# Altersfreigaben für unter 12 Jahre
under_12_ratings = ["G", "TV-Y", "TV-G", "TV-Y7", "TV-Y7-FV", "PG", "TV-PG"]

# Disney
print("Disney: Filme mit Altersfreigabe unter 12 Jahren")
print(disney[disney['rating'].isin(under_12_ratings)])

# Netflix
print("\nNetflix: Filme mit Altersfreigabe unter 12 Jahren")
print(netflix[netflix['rating'].isin(under_12_ratings)])


Disney: Filme mit Altersfreigabe unter 12 Jahren
     show_id  ...                                        description
0         s1  ...   Join Mickey and the gang as they duck the halls!
1         s2  ...  Santa Claus passes his magic bag to a new St. ...
2         s3  ...          Sid the Sloth is on Santa's naughty list.
3         s4  ...               This is real life, not just fantasy!
8         s9  ...  A day in the life at ZooTampa is anything but ...
...      ...  ...                                                ...
1442   s1443  ...  A jaded genius and an optimistic teen unearth ...
1443   s1444  ...  Two 13-year-old best friends embark on an adve...
1444   s1445  ...  Our prehistoric pals visit a world inhabited b...
1446   s1447  ...  Larry Daley returns to rescue some old friends...
1449   s1450  ...         View one of Sparky's favorite home movies.

[1302 rows x 10 columns]

Netflix: Filme mit Altersfreigabe unter 12 Jahren
     show_id  ...                             

# <a id='toc6_'></a>[4 Modelling](#toc0_)

## <a id='toc6_1_'></a>[](#toc0_)

es gibt bei Empfehlungssystemen einen kleinen Unterschied zu klassischen ML-Modellen:

Wir haben keine klassische Zielvariable (y) → Stattdessen nutzen wir Ähnlichkeitsmessungen.
Das bedeutet, dass ein normaler train_test_split nicht immer sinnvoll ist.

1. Train-Test-Split mit Scikit-Learn

Falls du trotzdem eine klassische Aufteilung möchtest:
Hier teilen wir das Feature-Set in 70% Training, 15% Validierung, 15% Test.

2. Alternativer Ansatz: Leave-One-Out-Cross-Validation (LOOCV)

Da wir Filme basierend auf Ähnlichkeit empfehlen, ist eine Leave-One-Out-Validierung (LOOCV) oft besser.
LOOCV simuliert echte Empfehlungen:
Nimmt einen Film als "Test"
Überprüft, ob ähnliche Filme richtig empfohlen wurden

wir sind an diesem Punkt angekommen. was sind die nächsten Schritte, damit der Ergebnis dieser Business case ist. : 

gebe mir den code mit kommentaren, Erklärungen und Kapitel Überschriften für den Jupiternotebook
 
Eingabe: 
- age: Droptown liste: 1, 2, 3, 4, 5, 6, ....bis 99 [muss] -> 100%
- hast du ein Abo: Netflix, Disney, beide -> nur eins aaswählbar [muss] -> 100%
- Anlass: 8 aufgezählt + 'no event' (bei no event werden events bei der suche nicht beachtet) -> mehrfach Auswahl möglich bei allen außer bei 'no event'[muss] -> 70%
- bevorzugte Genres: alle aufgezählt -> Mehrfachauswahl [kann, muss nicht] -> 40
- möchtest du angäbe zu etwas machen: liebkingsfilm, Lieblingsregisseur, Lieblingsschauspieler (Mehrfachauswahl möglich, nicht muss) wenn es angeklickt wird dann kommt eine section unterhalb um Vorschein  mit einem Textfeld